In [3]:
'''
20190125 增加分批抓取注释
'''

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time,re,requests
import pandas as pd

search_keyword="汶川地震"
browser = webdriver.Chrome()
################################################
#学外アクセス
def gakugai(browser,un,pw):
    browser.get("http://www.wul.waseda.ac.jp.ez.wul.waseda.ac.jp/DOMEST/db_about/pdo/pdo.html")
    elem_user = browser.find_element_by_name("user")
    elem_user.send_keys(un)
    elem_pwd = browser.find_element_by_name("pass")
    elem_pwd.send_keys(pw)
    browser.find_element_by_xpath('//*[@type="submit"]').click()
    time.sleep(1)
    browser.find_element_by_class_name("A_button").click()
    time.sleep(1)
    browser.switch_to_window(browser.window_handles[1])
    browser.get("http://data.people.com.cn.ez.wul.waseda.ac.jp/rmrb")
user="##########"
password="############"
#gakugai(browser,user,password)
################################################
#学内
browser.get("http://data.people.com.cn.ez.wul.waseda.ac.jp/rmrb")
################################################
#時期限定検索
def phase(browser,search_keyword,timeA,timeB):
    #browser.get("http://data.people.com.cn/rmrb/")
    browser.maximize_window()
    browser.find_element_by_xpath('//*[@value="2"and@name="dateTimeType"]').click()
    browser.find_element_by_name("dateTimeStart").send_keys(timeA)
    browser.find_element_by_name("dateTimeEnd").send_keys(timeB)
    browser.find_element_by_xpath('//*[@type="text"and@name="contentText"]').send_keys(search_keyword)
    browser.find_element_by_xpath('//*[@type="submit"and@class="input_s"]').click()
#ステップ①時期変更
starttime="2014-01-01"
endtime="2018-12-31"
phase(browser,search_keyword,starttime,endtime)

###############################################
#検索
#browser.get("http://data.people.com.cn/rmrb/")
#browser.find_element_by_name("queryStr").send_keys(search_keyword)
#browser.find_element_by_class_name("search_btn").click()
#browser.find_element_by_id("searchInAll").click()
browser.find_element_by_xpath('//*[@class="sortUl_li"and@value="1"]').click()
time.sleep(1)
################################################
#ステップ②記事番号更新
filenum="1184"
articles=[]

def onepage(browser,start,pdlist):
    linklist=[]
    for link in browser.find_elements_by_xpath('//*[@class="open_detail_link"]'):
        #htmldata=requests.get(link.get_attribute("href")).text
        getlink=link.get_attribute("href")
        linklist.append(getlink)
    for truelink in linklist:
        onettl,onedetail,oneset=getdetail(browser,truelink)        
        fn=start
        f=open("C:\\Users\\kanag\\Desktop\\Renminribao\\"+fn+".txt","w",encoding="utf-8")
        f.write("<title>"+onettl+"</title>")
        f.write("\n")
        f.write(onedetail)
        f.close()
        pdlist.append(oneset)
        start=str("%04d" % (int(start)+1))
        time.sleep(0.5)
    try:
        browser.find_element_by_xpath('//*[@title="下一页"]').click()
    except:
        pass
    return start

def getdetail(browser,url):
    #url.click()
    #browser.switch_to_window(browser.window_handles[1])
    browser.get(url)
    ttl=browser.find_element_by_class_name("title").text
    left=browser.find_element_by_class_name("sha_left").text
    date=re.search(r"(?<=人民日报)\d{4}年\d{1,2}月\d{1,2}日",left).group()
    area=re.search(r"第\d{1,2}版",left).group()
    try:
        category=re.search(r"(?<=版).+(?=】)",left).group()
    except:
        category=""
    detail=browser.find_element_by_class_name("detail_con").text
    #print(ttl)
    #print(detail)
    time.sleep(0.75)
    browser.back()
    time.sleep(0.25)
    return ttl,detail,[ttl,date,area,category]

#ページの切り替え
w_start=1
w_news=browser.find_element_by_xpath('//*[@id="allDataCount"]')
w_page=int(w_news.text)//20+1
while w_start<=w_page:
    filenum=onepage(browser,filenum,articles)
    w_start+=1
    
browser.quit()

#ステップ③CSVファイルネーム番号変更
df = pd.DataFrame(articles,columns=["タイトル","時間","版","カテゴリー"])
df.to_csv("Renminribao5.csv",encoding="utf-8",sep="\t", index=False)
df

,タイトル,時間,版,カテゴリー
0,行吟大九寨(美丽中国),2014年1月4日,第12版,副刊
1,公益慈善 莫论“出身”（深阅读）,2014年1月4日,第4版,要闻
2,以法治呵护公共精神成长（新论·学者说改革）,2014年1月6日,第5版,评论
3,谋利民之事 尽兴党之责,2014年1月18日,第6版,要闻
4,十年风雨炼成钢,2014年1月26日,第6版,国防周刊
5,各地变速换挡 发展更重质量,2014年1月30日,第10版,经济
6,三地马年“当头炮”,2014年2月15日,第1版,要闻
7,国法如天,2014年2月21日,第9版,视点
8,郭鑫 创业是希望的种子（100个人的中国梦·传递基层正能量）,2014年2月24日,第14版,连线基层
9,为昆明严重暴恐案遇难群众 政协开幕会上，集体默哀,2014年3月4日,第4版,要闻


In [5]:
import pandas as pd
import matplotlib.pyplot as plt

frames=[]
for datanum in range(1,6):
    filename="Renminribao"+str(datanum)+".csv"
    locals()["df"+str(datanum)]=pd.read_csv(filename,encoding="utf-8", sep="\t",engine="python")
    frames.append(locals()["df"+str(datanum)])

result = pd.concat(frames)
result.to_excel('Renminribao.xlsx', sheet_name='sheet1')
#result.to_csv("Renminribao.csv",encoding="utf-8",sep="\t", index=False)
result

,タイトル,時間,版,カテゴリー
0,抗震救灾款物基本合理使用,2009年1月1日,第3版,要闻
1,对地震灾区灾前贷款再次给予宽限期,2009年1月2日,第3版,要闻
2,中央临时生活救助资金发放工作基本完成,2009年1月5日,第1版,要闻
3,中央临时生活救助资金发放工作基本完成,2009年1月5日,第2版,国内要闻
4,为社会主义新农村建设提供保障,2009年1月5日,第4版,要闻
5,震区重建四记,2009年1月5日,第5版,视点新闻
6,震区六成以上倒损农房开工重建,2009年1月7日,第1版,要闻
7,四川汶川地震灾区学生将应邀访问菲律宾,2009年1月9日,第4版,要闻
8,确保全部救灾款物真正用于灾区和受灾群众,2009年1月10日,第6版,综合
9,廉租房建设驶入快车道,2009年1月11日,第1版,要闻


In [6]:
import os,re

class News():
    def __init__(self,read):
        try:
            self.ttl=re.search(r"(?<=<title>).+(?=</title>)",read).group()
            self.ctt=re.sub(r"<title>.+</title>\n","",read)
            self.khcoder="<h1>"+self.ttl+"</h1>\n"+self.ctt
        except:
            self.ttl,self.ctt,self.khcoder="","",""
        
    def __str__(self):
        output_for_python=self.ttl+"\n"+self.ctt
        return output_for_python

path ="Desktop/Renminribao"
files= os.listdir(path)

fopen=open("Desktop/Renminribao/renminribao_text.txt","w",encoding="utf-8")

for file in files:
    if not os.path.isdir(file):
        fr=open(path+"/"+file,"r",encoding="utf-8")
        fn=News(fr.read())
        fr.close()
        fopen.write(fn.khcoder)

fopen.close()
print("Done")

Done
